In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

##########

import math
import statistics

##########

plt.rcParams['figure.figsize'] = [20,10]

##########

def parse_ndarray(s):
    return np.fromstring(s, sep=' ') if s else None

def running_avg(x):
    return np.cumsum(x) / np.arange(1, x.size + 1)

def running_timeavg(t,x):
    dt = t[1:] - t[:-1]
    return np.cumsum(x[:-1] * dt) / t[1:]

##########

ts90 = 1.64
ts95 = 1.96

##########

def orderrunlist(mylist):
    orderedlist = [mylist[0]]
    for i in range(11, 20):
        orderedlist.append(mylist[i])
    for i in range(1, 11):
        orderedlist.append(mylist[i])
    return orderedlist;

def computevals(vallist):
    mean = statistics.mean(vallist)
    var = (1 / (len(vallist) - 1)) * sum([(v - mean)**2 for v in vallist])
    stdev = math.sqrt(var)
    confinterval = [[mean - ts90 * (stdev / math.sqrt(len(vallist))), mean + ts90 * (stdev / math.sqrt(len(vallist)))], [mean - ts95 * (stdev / math.sqrt(len(vallist))), mean + ts95 * (stdev / math.sqrt(len(vallist)))]]
    return [mean, var, stdev, confinterval]

In [ ]:
# Analisi del numero di rilevazioni effettuate per verificare l'attendibilità dei risultati

# Job creati dalla sorgente
sourcedata = list()
# Job terminati nel pozzo
sinkdata = list()

for i in range(48):
    n = pd.read_csv('csvresults/n{}.csv'.format(i + 1))
    
    # Ricerca e ordinamento dei Job generati dalla sorgente
    sodraw = list()
    tmp = n[(n['module'] == 'Network.source') & (n['name'] == 'created:last') & (n['type'] == 'scalar')]
    for t in tmp.itertuples():
        sodraw.append(t.value)
    sod = orderrunlist(sodraw)
    for s in sod:
        sourcedata.append(int(s))
    
    # Ricerca e ordinamento dei Job terminati nel pozzo
    sidraw = list()
    tmp = n[(n['module'] == 'Network.sink') & (n['name'] == 'lifeTime:vector') & (n['type'] == 'vector')]
    for t in tmp.itertuples():
        sidraw.append(len(t.vecvalue.split(' ')))
    sid = orderrunlist(sidraw)
    for s in sid:
        sinkdata.append(int(s))

sn = 0
nsm = 0
ms = 0
for s in sourcedata:
    if s < 900:
        sn = sn + 1
    elif s < 1000:
        nsm = nsm + 1
    else:
        ms = ms + 1
print('=== Job generati ===')
print('numero di esperimenti : {}'.format(len(sourcedata)))
print('osservazioni >= 1000 : {} => {:%}'.format(ms, ms / len(sourcedata)))
print('900 <= osservazioni < 1000 : {} => {:%}'.format(nsm, nsm / len(sourcedata)))
print('osservazioni < 900 : {} => {:%}'.format(sn, sn / len(sourcedata)))

sn = 0
nsm = 0
ms = 0
for s in sinkdata:
    if s < 900:
        sn = sn + 1
    elif s < 1000:
        nsm = nsm + 1
    else:
        ms = ms + 1
print('\n=== Job terminati ===')
print('numero di esperimenti : {}'.format(len(sinkdata)))
print('osservazioni >= 1000 : {} => {:%}'.format(ms, ms / len(sinkdata)))
print('900 <= osservazioni < 1000 : {} => {:%}'.format(nsm, nsm / len(sinkdata)))
print('osservazioni < 900 : {} => {:%}'.format(sn, sn / len(sinkdata)))

print('\n=== sourcedata ===')
print(sourcedata)

print('\n=== sinkdata ===')
print(sinkdata)

In [ ]:
# Analisi delle rilevazioni rimanenti escludendo il transiente iniziale

# Job terminati nel pozzo
sinkdata200 = list()
sinkdata300 = list()
sinkdata400 = list()

for i in range(48):
    n = pd.read_csv('csvresults/n{}.csv'.format(i + 1))
    # Ricerca e ordinamento dei Job terminati nel pozzo
    sidraw = list()
    tmp = n[(n['module'] == 'Network.sink') & (n['name'] == 'lifeTime:vector') & (n['type'] == 'vector')]
    for t in tmp.itertuples():
        sidraw.append(t.vectime)
    sid = orderrunlist(sidraw)
    for s in sid:
        sid200 = [float(f) for f in s.split(' ') if float(f) >= 200]
        sinkdata200.append(int(len(sid200)))
        sid300 = [float(f) for f in s.split(' ') if float(f) >= 300]
        sinkdata300.append(int(len(sid300)))
        sid400 = [float(f) for f in s.split(' ') if float(f) >= 400]
        sinkdata400.append(int(len(sid400)))

s5 = 0
s56 = 0
s67 = 0
s78 = 0
s89 = 0
s91 = 0
s1 = 0
for s in sinkdata200:
    if s < 500:
        s5 = s5 + 1
    elif s < 600:
        s56 = s56 + 1
    elif s < 700:
        s67 = s67 + 1
    elif s < 800:
        s78 = s78 + 1
    elif s < 900:
        s89 = s89 + 1
    elif s < 1000:
        s91 = s91 + 1
    else:
        s1 = s1 + 1
print('=== Job terminati [s >= 200]===')
print('numero di esperimenti : {}'.format(len(sinkdata200)))
print('osservazioni >= 1000 : {} => {:%}'.format(s1, s1 / len(sinkdata200)))
print('900 <= osservazioni < 1000 : {} => {:%}'.format(s91, s91 / len(sinkdata200)))
print('800 <= osservazioni < 900 : {} => {:%}'.format(s89, s89 / len(sinkdata200)))
print('700 <= osservazioni < 800 : {} => {:%}'.format(s78, s78 / len(sinkdata200)))
print('600 <= osservazioni < 700 : {} => {:%}'.format(s67, s67 / len(sinkdata200)))
print('500 <= osservazioni < 600 : {} => {:%}'.format(s56, s56 / len(sinkdata200)))
print('osservazioni < 500 : {} => {:%}'.format(s5, s5 / len(sinkdata200)))

s5 = 0
s56 = 0
s67 = 0
s78 = 0
s89 = 0
s91 = 0
s1 = 0
for s in sinkdata300:
    if s < 500:
        s5 = s5 + 1
    elif s < 600:
        s56 = s56 + 1
    elif s < 700:
        s67 = s67 + 1
    elif s < 800:
        s78 = s78 + 1
    elif s < 900:
        s89 = s89 + 1
    elif s < 1000:
        s91 = s91 + 1
    else:
        s1 = s1 + 1
print('\n=== Job terminati [s >= 300]===')
print('numero di esperimenti : {}'.format(len(sinkdata300)))
print('osservazioni >= 1000 : {} => {:%}'.format(s1, s1 / len(sinkdata300)))
print('900 <= osservazioni < 1000 : {} => {:%}'.format(s91, s91 / len(sinkdata300)))
print('800 <= osservazioni < 900 : {} => {:%}'.format(s89, s89 / len(sinkdata300)))
print('700 <= osservazioni < 800 : {} => {:%}'.format(s78, s78 / len(sinkdata300)))
print('600 <= osservazioni < 700 : {} => {:%}'.format(s67, s67 / len(sinkdata300)))
print('500 <= osservazioni < 600 : {} => {:%}'.format(s56, s56 / len(sinkdata300)))
print('osservazioni < 500 : {} => {:%}'.format(s5, s5 / len(sinkdata300)))

s5 = 0
s56 = 0
s67 = 0
s78 = 0
s89 = 0
s91 = 0
s1 = 0
for s in sinkdata400:
    if s < 500:
        s5 = s5 + 1
    elif s < 600:
        s56 = s56 + 1
    elif s < 700:
        s67 = s67 + 1
    elif s < 800:
        s78 = s78 + 1
    elif s < 900:
        s89 = s89 + 1
    elif s < 1000:
        s91 = s91 + 1
    else:
        s1 = s1 + 1
print('\n=== Job terminati [s >= 400]===')
print('numero di esperimenti : {}'.format(len(sinkdata400)))
print('osservazioni >= 1000 : {} => {:%}'.format(s1, s1 / len(sinkdata400)))
print('900 <= osservazioni < 1000 : {} => {:%}'.format(s91, s91 / len(sinkdata400)))
print('800 <= osservazioni < 900 : {} => {:%}'.format(s89, s89 / len(sinkdata400)))
print('700 <= osservazioni < 800 : {} => {:%}'.format(s78, s78 / len(sinkdata400)))
print('600 <= osservazioni < 700 : {} => {:%}'.format(s67, s67 / len(sinkdata400)))
print('500 <= osservazioni < 600 : {} => {:%}'.format(s56, s56 / len(sinkdata400)))
print('osservazioni < 500 : {} => {:%}'.format(s5, s5 / len(sinkdata400)))

print('\n=== sinkdata200 ===')
print(sinkdata200)

print('\n=== sinkdata300 ===')
print(sinkdata300)

print('\n=== sinkdata400 ===')
print(sinkdata400)

In [ ]:
# Network.sink.lifeTime:vector [mediana] [min] [max] [mean]

# Valore da cui escludere il transiente iniziale
transiente = 300
# Mediane di ogni configurazione
medianas = list()
# Minimi di ogni configurazione
minimums = list()
# Massimi di ogni configurazione
maximums = list()
# Media di ogni configurazione
means = list()

for i in range(48):
    n = pd.read_csv('csvresults/n{}.csv'.format(i + 1))
    # Ricerca dei vettori
    veclistraw = list()
    vectors = n[(n['module'] == 'Network.sink') & (n['name'] == 'lifeTime:vector') & (n['type'] == 'vector')]
    for v in vectors.itertuples():
        veclistraw.append(v)
    # Ordinamento dei run
    veclist = orderrunlist(veclistraw)
    # Mediane dei run
    mediana = list()
    # Minimi dei run
    minimum = list()
    # Massimi dei run
    maximum = list()
    # Medie dei run
    mean = list()
    for v in veclist:
        # Esclusione del transiente iniziale
        vtrans = [float(f) for f in v.vectime.split(' ') if float(f) >= transiente]
        sortedv = sorted(v.vecvalue.split(' ')[-len(vtrans):])
        # Calcolo della mediana
        mediana.append(float(sortedv[int(len(sortedv) / 2)]))
        # Calcolo del minimo
        minimum.append(float(sortedv[0]))
        # Calcolo del massimo
        maximum.append(float(sortedv[-1]))
        # Calcolo della media
        mean.append(statistics.mean([float(f) for f in sortedv]))
        
    medianas.append(mediana)
    minimums.append(minimum)
    maximums.append(maximum)
    means.append(mean)

# Valori delle mediane
medianaval = list()
# Valori dei minimi
minimumval = list()
# Valori dei massimi
maximumval = list()
# Valori delle medie
meanval = list()

for m in medianas:
    medianaval.append(computevals(m))
    
for m in minimums:
    minimumval.append(computevals(m))
    
for m in maximums:
    maximumval.append(computevals(m))
    
for m in means:
    meanval.append(computevals(m))
    
print('=== Mediana ===')
print(medianaval)

print('\n=== Minimo ===')
print(minimumval)

print('\n=== Massimo ===')
print(maximumval)

print('\n=== Media ===')
print(meanval)

In [ ]:
# Network.server.droppedForDeadline:count

# Droppati di ogni configurazione
drops = list()

for i in range(48):
    n = pd.read_csv('csvresults/n{}.csv'.format(i + 1))
    # Ricerca degli scalari
    scalistraw = list()
    scalars = n[(n['module'] == 'Network.server') & (n['name'] == 'droppedForDeadline:count') & (n['type'] == 'scalar')]
    for s in scalars.itertuples():
        scalistraw.append(s.value)
    # Ordinamento dei run
    scalist = orderrunlist(scalistraw)
    # Medie dei run
    drops.append(scalist)

# Valori delle medie
dropval = list()
    
for d in drops:
    dropval.append(computevals(d))

print('\n=== Droppati ===')
print(dropval)

In [ ]:
# Grafici per Network.sink.lifeTime:vector [medie]

for i in range(48):
    n = pd.read_csv('csvresults/n{}.csv'.format(i + 1), converters = {'vectime': parse_ndarray, 'vecvalue': parse_ndarray})

    vectors = n[(n['module'] == 'Network.sink') & (n['name'] == 'lifeTime:vector') & (n['type'] == 'vector')]
    
    for v in vectors.itertuples():
        plt.plot(v.vectime, running_avg(v.vecvalue))
    plt.title('Configurazione {}, Network.sink.lifeTime:vector [medie]'.format(i + 1))
    plt.show()

In [ ]:
# Grafici per Network.passiveQueue*.queueLength:vector [medie]

for i in range(48):
    n = pd.read_csv('csvresults/n{}.csv'.format(i + 1), converters = {'vectime': parse_ndarray, 'vecvalue': parse_ndarray})
    
    vectors = n[(n['module'] == 'Network.passiveQueue') & (n['name'] == 'queueLength:vector') & (n['type'] == 'vector')]
    vectors1 = n[(n['module'] == 'Network.passiveQueue1') & (n['name'] == 'queueLength:vector') & (n['type'] == 'vector')]
    vectors2 = n[(n['module'] == 'Network.passiveQueue2') & (n['name'] == 'queueLength:vector') & (n['type'] == 'vector')]
    vectors3 = n[(n['module'] == 'Network.passiveQueue3') & (n['name'] == 'queueLength:vector') & (n['type'] == 'vector')]
    
    for v in vectors.itertuples():
        plt.plot(v.vectime, running_avg(v.vecvalue))
    for v in vectors1.itertuples():
        plt.plot(v.vectime, running_avg(v.vecvalue))
    for v in vectors2.itertuples():
        plt.plot(v.vectime, running_avg(v.vecvalue))
    for v in vectors3.itertuples():
        plt.plot(v.vectime, running_avg(v.vecvalue))
    plt.title('Configurazione {}, Network.passiveQueue*.queueLength:vector [medie]'.format(i + 1))
    plt.show()